In [ ]:
import panel as pn
from ipyleaflet import Map, basemaps, DrawControl, AwesomeIcon, Marker
from datetime import date
import os
import pandas as pd
from io import StringIO
pn.extension("ipywidgets", sizing_mode="stretch_width")

tabs = pn.Tabs(active=2, tabs_location="left")

In [ ]:
## Create widgets.
headings = pn.pane.Markdown('''
## Approximately how many seconds have you lived?
### Enter your information here and see!
''')
name = pn.widgets.TextInput(name="Name", value="Venus")
age = pn.widgets.IntSlider(name="Age", value=50, start=1, end=100)
color = pn.widgets.ColorPicker(name="Color", value="#4f4fdf")
calc_button = pn.widgets.Button(name="Calculate", button_type="primary")
result = pn.widgets.StaticText()

## Create functions.
def compute_secs_lived(event):
  secs = age.value * 365 * 24 * 60 * 60
  result.value = f"Hello {name.value}, you have lived for approximately {secs} seconds!"
  result.style = {"color": color.value}
  return result

## Create interactions between widgets and functions.
calc_button.on_click(compute_secs_lived)

## Create person component and add it as a tab.
person_inputs = pn.Row(name, age, color)
person_component = pn.Column(headings, person_inputs, calc_button, result)
tabs.append(("Person", person_component))

In [ ]:
## Create map widgets.
# center = [vertical distance on y-axis, horizontal distance on x-axis]
center = [37, -122]
map = Map(basemap=basemaps.OpenStreetMap.France, center=center, zoom=10)
icon = AwesomeIcon(
  name="heart",
  marker_color="blue",
  icon_color="plum",
  spin=True
)
marker = Marker(icon=icon, location=(center[0] - 0.035, center[1] - 0.015))
map.add_layer(marker)

display_geo_button = pn.widgets.Button(name="Display Geometry", button_type="success")
clear_drawings_button = pn.widgets.Button(name="Clear Drawings")

geo = ""
drawing_geometry = pn.widgets.StaticText(name="Last Drawing's Geometry", value=geo)
count = 0
num_drawings = pn.widgets.StaticText(name="Number of Drawings", value=count)

## Create and customize drawing controls.
draw_control = DrawControl(polyline={}, polygon={}, circle={}, circlemarker={})
draw_control.rectangle = {
  "shapeOptions": {
    "fillColor": "#4f4fdf",
    "color": "#4f4fdf",
    "fillOpacity": 0.3
  }
}
map.add_control(draw_control)

# Save drawn rectangle and increment number of drawings.
def save_drawing(self, action, geo_json):
  global geo, count
  geo = geo_json["geometry"]
  count += 1

# Display geometry of recently drawn rectangle and total number of drawings.
def print_drawing_geometry(event):
  # drawing_geometry.value = draw_control.last_draw["geometry"]
  drawing_geometry.value = geo
  num_drawings.value = count // 2

# Clear all drawings and reset drawing count.
def clear_drawings(event):
  draw_control.clear()
  global geo, count
  geo = ""
  drawing_geometry.value = geo
  count = 0
  num_drawings.value = count

## Create interactions between widgets and functions.
draw_control.on_draw(save_drawing)
display_geo_button.on_click(print_drawing_geometry)
clear_drawings_button.on_click(clear_drawings)

## Create map component and add it as a tab.
map_buttons = pn.Row(display_geo_button, clear_drawings_button)
map_component = pn.Column(map, map_buttons, drawing_geometry, num_drawings)
tabs.append(("Map", map_component))

In [ ]:
## Create CoastSeg controls with Panel widgets.
date_heading = pn.pane.Markdown('''
## Pick a date:
''')
start_date_picker = pn.widgets.DatePicker(name="Start Date", value=date(2018,12,1))
end_date_picker = pn.widgets.DatePicker(name="End Date", value=date(2019,3,1))
collection_heading = pn.pane.Markdown('''
## Pick a collection:
C01: LandSat Collection 1 (all images prior to 2022-01-01)\n
C02: LandSat Collection 2 (all images after to 2022-01-01)\n
- LandSat 9 is only available in C02
''')
collection_radio_button_group = pn.widgets.RadioButtonGroup(name="Collection", options=["C01", "C02"], button_type="primary", orientation="vertical")
# collection_radio_box_group = pn.widgets.RadioBoxGroup(name="Collection", options=["C01", "C02"], inline=False)
satellites_heading = pn.pane.Markdown('''
## Pick at least one satellite:
Unselect a satellite by clicking on the option again.
''')
satellites_check_button_group = pn.widgets.CheckButtonGroup(name="Satellites", options=["L5", "L7", "L8", "S2"], value=["L8"], button_type="warning")
# satellites_multi_select = pn.widgets.MultiSelect(name="Satellites", options=["L5", "L7", "L8", "S2"], value=["L8"])

## Create CoastSeg controls component and add it as a tab.
date_pickers = pn.Row(start_date_picker, end_date_picker)
coastseg_component = pn.Column(date_heading, date_pickers, collection_heading, collection_radio_button_group, satellites_heading, satellites_check_button_group)
tabs.append(("CoastSeg", coastseg_component))

In [ ]:
## Create data widgets.
data_files = os.listdir("data")
file_select = pn.widgets.Select(name="File", options=data_files, value=data_files[0])

# Binds file_select widget's selected value to selected_filename argument.
@pn.depends(file_select)
def get_selected_file_path(selected_filename):
  return "data/" + selected_filename

selected_file_download = pn.widgets.FileDownload(callback=get_selected_file_path, filename=file_select.value, button_type="success", align="end")

# Link value parameter of source (file_select widget) to filename parameter of target (selected_file_download button).
file_select.link(selected_file_download, value="filename")

data = pd.read_csv("data/A_naip_meta_served.csv")
data = data.set_index("label_image_filename")
years_options = list(data.year.unique())
years_multi_choice = pn.widgets.MultiChoice(name="Years", options=years_options, value=[years_options[0]], solid=False)
doodle_spatial_density_range_slider = pn.widgets.RangeSlider(name="Proportion of Image Annotated", start=data.doodle_spatial_density.min(), end=data.doodle_spatial_density.max(), bar_color="#4f4fdf")

# Filter data based on doodle_spatial_density.
@pn.depends(years_multi_choice, doodle_spatial_density_range_slider)
def get_filtered_data(years, density):
  global data
  filtered_data = data
  [min_density, max_density] = density
  if years_multi_choice.value:
    filtered_data = filtered_data[filtered_data.year.isin(years)]
  return filtered_data[(filtered_data.doodle_spatial_density >= min_density) & (filtered_data.doodle_spatial_density <= max_density)]

# Saves filtered data's DataFrame object as a CSV, and converts it into a file StringIO object.
@pn.depends(years_multi_choice, doodle_spatial_density_range_slider)
def get_filtered_file(ymc, dsdrs):
  dataframe = get_filtered_data(ymc, dsdrs)
  sio = StringIO()
  dataframe.to_csv(sio)
  sio.seek(0)
  return sio

filtered_data_file_download = pn.widgets.FileDownload(callback=get_filtered_file, filename="filtered_data.csv", button_type="primary")

## Create data component and add it as a tab.
download_selected_file = pn.Row(file_select, selected_file_download)
filter_controls = pn.Row(years_multi_choice, doodle_spatial_density_range_slider)
info_text = """
  ### Filtered Data
  The following data is a preview of the content in `filtered_data.csv`:
"""
data_component = pn.Column("## Select a File to Download", download_selected_file, "## Filter Data to Download", filter_controls, filtered_data_file_download, pn.pane.Alert(info_text, alert_type="info"), pn.panel(get_filtered_data))
tabs.append(("Data", data_component))

In [ ]:
## Add components to serve on app.
app = pn.Column("# My First Panel App!", tabs)
app.servable()